In [1]:
import sys
sys.path.append("..")
from download_utils import download_project_resources

download_project_resources()

File data\dialogues.tsv is already downloaded.
File data\tagged_posts.tsv is already downloaded.


In [3]:
from utils import *

In [4]:
import numpy as np
import pandas as pd
import pickle
import re
import utils

from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
def tfidf_features(X_train, X_test, vectorizer_path):
    """Performs TF-IDF transformation and dumps the model."""
    
    # Train a vectorizer on X_train data.
    # Transform X_train and X_test data.
    
    # Pickle the trained vectorizer to 'vectorizer_path'
    # Don't forget to open the file in writing bytes mode.
    
    tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1, 2),
                                       token_pattern='(\S+)')
    X_train=tfidf_vectorizer.fit_transform(X_train)
    X_test=tfidf_vectorizer.transform(X_test)
    with open(vectorizer_path,'wb') as vectorizer_file: 
        pickle.dump(tfidf_vectorizer,vectorizer_file)
    
    return X_train, X_test

Now, load examples of two classes. Use a subsample of stackoverflow data to balance the classes. You will need the full data later.

In [6]:
sample_size = 200000

dialogue_df = pd.read_csv('data/dialogues.tsv', sep='\t').sample(sample_size, random_state=0)
stackoverflow_df = pd.read_csv('data/tagged_posts.tsv', sep='\t').sample(sample_size, random_state=0)

Check how the data look like:

In [7]:
dialogue_df.head()

,text,tag
82925,"Donna, you are a muffin.",dialogue
48774,He was here last night till about two o'clock....,dialogue
55394,"All right, then make an appointment with her s...",dialogue
90806,"Hey, what is this-an interview? We're supposed...",dialogue
107758,Yeah. He's just a friend of mine I was trying ...,dialogue


In [8]:
stackoverflow_df.head()

,post_id,title,tag
2168983,43837842,Efficient Algorithm to compose valid expressio...,python
1084095,15747223,Why does this basic thread program fail with C...,c_cpp
1049020,15189594,Link to scroll to top not working,javascript
200466,3273927,Is it possible to implement ping on windows ph...,c#
1200249,17684551,GLSL normal mapping issue,c_cpp


In [9]:
from utils import RESOURCE_PATH

In [10]:
dialogue_df['text'] = dialogue_df['text'].apply(text_prepare)
stackoverflow_df['title'] = stackoverflow_df['title'].apply(text_prepare)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X = np.concatenate([dialogue_df['text'].values, stackoverflow_df['title'].values])
y = ['dialogue'] * dialogue_df.shape[0] + ['stackoverflow'] * stackoverflow_df.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

X_train_tfidf, X_test_tfidf = tfidf_features(X_train, X_test,'./tfidf_vectorizer.pkl')

Train size = 360000, test size = 40000


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [14]:
intent_recognizer=LogisticRegression(solver='newton-cg',C=10, penalty='l2',n_jobs=-1)
intent_recognizer.fit(X_train_tfidf, y_train)

LogisticRegression(C=10, n_jobs=-1, solver='newton-cg')

In [16]:
# Check test accuracy.
y_test_pred = intent_recognizer.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.991575


In [23]:
pickle.dump(intent_recognizer, open(RESOURCE_PATH['INTENT_RECOGNIZER1'], 'wb'))

In [19]:
X = stackoverflow_df['title'].values
y = stackoverflow_df['tag'].values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

Train size = 160000, test size = 40000


In [32]:
vectorizer = pickle.load(open('tfidf_vectorizer.pkl', 'rb'))

X_train_tfidf, X_test_tfidf = vectorizer.transform(X_train), vectorizer.transform(X_test)

In [33]:
from sklearn.multiclass import OneVsRestClassifier

In [34]:
lr = LogisticRegression(solver='newton-cg',C=5, penalty='l2',n_jobs=-1)
tag_classifier = OneVsRestClassifier(lr)
tag_classifier.fit(X_train_tfidf, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=5, n_jobs=-1,
                                                 solver='newton-cg'))

In [35]:
# Check test accuracy.
y_test_pred = tag_classifier.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.8007


In [36]:
pickle.dump(tag_classifier, open(RESOURCE_PATH['TAG_CLASSIFIER'], 'wb'))

In [38]:
starspace_embeddings, embeddings_dim = load_embeddings('word_embeddings.tsv')

In [39]:
posts_df = pd.read_csv('data/tagged_posts.tsv', sep='\t')

In [32]:
counts_by_tag = posts_df.groupby(posts_df['tag']).count()
counts_by_tag.items()

<generator object DataFrame.items at 0x000001DE0FF28840>

In [40]:
counts_by_tag = posts_df['tag'].value_counts().to_dict()

In [41]:
import os
os.makedirs(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], exist_ok=True)

for tag, count in counts_by_tag.items():
    tag_posts = posts_df[posts_df['tag'] == tag]
    
    tag_post_ids = tag_posts['post_id'].values
    
    tag_vectors = np.zeros((count, embeddings_dim), dtype=np.float32)
    for i, title in enumerate(tag_posts['title']):
        tag_vectors[i, :] = question_to_vec(title,starspace_embeddings, embeddings_dim)

    # Dump post ids and vectors to a file.
    filename = os.path.join(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], os.path.normpath('%s.pkl' % tag))
    pickle.dump((tag_post_ids, tag_vectors), open(filename, 'wb'))

In [42]:
import utils
text_prepare("hello l-t-p-c s.t.a.c 4554 --  2019-20 + \ gen/obc + c++ +#@. {bj } | ")

'hello ltpc stac 4554 201920 + gen obc + c++ +# bj'